# Continuous Control

In this notebook we will train a Deep Deterministic Policy Gradient (DDPG) Agent to control a double-jointed arm to move and reach target locations.

In [ ]:
import numpy as np
import torch
from ddpg.agent import Agent
from unityagents import UnityEnvironment
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

### Check for GPU device. If not available use CPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

### Create the environment and set the default for controlling

Please change the file name of the environment in case the file path or name is different

In [ ]:
env = UnityEnvironment(file_name="Reacher_Linux/Reacher.x86_64")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### Check the environment for Action and State size

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]

In [ ]:
action_size = brain.vector_action_space_size
state_size = len(state)

### Create an agent

In [ ]:
configs = {
    'seed': 0,
    'device': device,
    'lr_actor': 1e-4,
    'lr_critic': 1e-3,
    'buffer_size': 100000,
    'batch_size': 128,
    'gamma': 0.99,
    'tau': 1e-3
}

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, config=configs)

#### Helper method to train the agent on the environment
**Parameters:**

- **n_episodes(int)**: Maximum number of episodes to run the training process
- **max_t(int)**: Maximum number of time step the agent will spend in each episode
- **eps_start(float)**: Initial value of epsilon
- **eps_end(float)**: Minimum value of epsilon
- **eps_decay(float)**: The decay rate of epsilon for each episode
- **max_score(float)**: The avg. score in last 100 episodes after which the environment will be considered as solved
- **model_path(str)**: Path to save the trained model

In [ ]:
def train(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, max_score=30.0, model_path='model.pt'):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps*eps_decay)
        print(f"\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window)}", end="")
        if i_episode % 100 == 0:
            print(f"\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window)}")
        if np.mean(scores_window) >= max_score:
            print(f"\nEnvironment solved in {i_episode-100} episodes!\tAverage Score: {np.mean(scores_window)}")
            torch.save(agent.actor_local.state_dict(), model_path)
            break
    return scores

### Helper method to visualize the training scores
**Parameters:**

- **scores(list of floats)**: The scores collected for all episodes

In [ ]:
def visualize(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Scores')
    plt.xlabel('Episode')
    plt.show()

### Helper method to test a trained agent on the environment
**Parameters**

- **env**: The environmnts
- **agent**: The agent object
- **n_episodes(int)**: Number of episodes to run the test
- **model_path**: Path to a saved pytorch model

In [ ]:
def test(env, agent, n_episodes=3, model_path='model.pt'):
    agent.actor_local.load_state_dict(torch.load(model_path))
    total_score = 0
    for i_episode in range(1, n_episodes+1):
        score = 0
        env_info = env.reset(train_mode=False)[brain_name]
        state = env_info.vector_observations[0]
        while True:
            action = agent.act(state)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            score += reward
            state = next_state
            if done:
                break
        print(f"\rEpisode {i_episode} \tScore: {score}")
        total_score += score
    print(f"\rAverage score in {n_episodes} episodes: {total_score/n_episodes}")

### Run the training process

Start training the agent and visualize the scores after the completion of the training. In case of any error close the environment.

In [ ]:
try:
    scores = train()
    visualize(scores)
except Exception as e:
    env.close()
    raise e

### Test the agent
Use the trained agent to play the game.

In [ ]:
test(env=env, agent=agent)

### Close the environment after everything is done

In [ ]:
env.close()